In [ ]:
!pip install lxml

In [28]:
from pathlib import Path
import json
from datetime import datetime
import re
from tqdm import tqdm
data_dir = Path('/home/vbs/vbs23/data/vbs22-meta/')

In [29]:
list(data_dir.iterdir())[:3]

[PosixPath('/home/vbs/vbs23/data/vbs22-meta/infoV3C2'),
 PosixPath('/home/vbs/vbs23/data/vbs22-meta/infoV3C1')]

In [30]:
json_example = '/home/vbs/vbs23/data/vbs22-meta/infoV3C1/00001.json'
import json 
from pprint import pprint
pprint(json.load(open(json_example)))

{'categories': ['/categories/sports'],
 'channel': 'valentinxflad',
 'description': '<p class="first">On Sunday, May 25th, many riders from Paris '
                'and from Argentina came to the infamous &quot;Opera '
                'Spot&quot; in the middle of Paris to ride together.</p>\n'
                '<p>It was a great time: we rode as hard as we could, we spoke '
                'together, we chilled a lot...</p>\n'
                '<p>Valentin FLAD - May 2014</p>',
 'duration': 273,
 'height': 720,
 'license': 'by-nc-sa',
 'tags': ['fish eye',
          'edit',
          'flad',
          'vid?o',
          'acrobatique',
          'Flatland',
          'riding',
          'BMX',
          'canon',
          'v?lo',
          'paris',
          'jam',
          'Flat',
          'france',
          'sport',
          'Opera',
          'Bike',
          'extreme'],
 'title': 'PARISIANA JAM',
 'uploadDate': '2014-05-31 10:32:38',
 'v3cId': '00001',
 'vimeoId': '96990666',
 'w

In [31]:
!cat db_schema_old.py

cat: db_schema_old.py: No such file or directory


In [32]:
from db_schema_vbs22 import Video
import pydantic
json_valid = pydantic.parse_file_as(path=json_example, type_=Video)
json_valid

Video(v3cId='00001', vimeoId='96990666', title='PARISIANA JAM', description='<p class="first">On Sunday, May 25th, many riders from Paris and from Argentina came to the infamous &quot;Opera Spot&quot; in the middle of Paris to ride together.</p>\n<p>It was a great time: we rode as hard as we could, we spoke together, we chilled a lot...</p>\n<p>Valentin FLAD - May 2014</p>', duration=273, uploadDate='2014-05-31 10:32:38', channel='valentinxflad', license='by-nc-sa', width=1280, height=720, tags=['fish eye', 'edit', 'flad', 'vid?o', 'acrobatique', 'Flatland', 'riding', 'BMX', 'canon', 'v?lo', 'paris', 'jam', 'Flat', 'france', 'sport', 'Opera', 'Bike', 'extreme'], categories=['/categories/sports'])

In [33]:
from bs4 import BeautifulSoup

def clean_html(raw_html):
    return BeautifulSoup(raw_html, "lxml").text
print(json_valid.description)
print('-'*10)
print(clean_html(json_valid.description))

<p class="first">On Sunday, May 25th, many riders from Paris and from Argentina came to the infamous &quot;Opera Spot&quot; in the middle of Paris to ride together.</p>
<p>It was a great time: we rode as hard as we could, we spoke together, we chilled a lot...</p>
<p>Valentin FLAD - May 2014</p>
----------
On Sunday, May 25th, many riders from Paris and from Argentina came to the infamous "Opera Spot" in the middle of Paris to ride together.
It was a great time: we rode as hard as we could, we spoke together, we chilled a lot...
Valentin FLAD - May 2014


In [34]:
from pysearch.utils.time import month2month_of_year, day2day_of_week, hour2part_of_day

In [35]:
result = {}

for data_part_dir in data_dir.iterdir():
    pbar = list(data_part_dir.glob('*.json'))
    pbar = tqdm(pbar, total=len(pbar))
    for json_file in pbar:
        json_valid = pydantic.parse_file_as(path=json_file, type_=Video, encoding='Latin-1')
        shot_name = int(json_file.stem)

        # v3cId: str
        # vimeoId: str
        # title: str
        # description: str
        # duration: int
        # uploadDate: str
        # channel: str
        # license: str
        # width: int
        # height: int
        # tags: List[str]
        # categories: List[str]
        assert int(shot_name) == int(json_valid.v3cId), f'Invalid shot name: {shot_name}'
        _time = datetime.strptime(json_valid.uploadDate, '%Y-%m-%d %H:%M:%S')
        item = {
            'id': json_valid.v3cId,
            'vimeo_id': json_valid.vimeoId,
            'url': f'https://vimeo.com/{json_valid.vimeoId}',
            'title': json_valid.title,
            'description': clean_html(json_valid.description),
            'channel': json_valid.channel,
            'tags': json_valid.tags,
            'categories': json_valid.categories,
            'upload_date': json_valid.uploadDate,
            'date': _time.date().strftime('%Y%m%d'), # basic date format (for pysearch)
            'month': month2month_of_year(_time.month),
            'year': _time.year,
            'day_of_week': day2day_of_week(_time.weekday()),
            'part_of_day': hour2part_of_day(_time.hour),
            'local_time': _time.time().strftime('%H:%M'),
            'width': json_valid.width,
            'height': json_valid.height,
        }
        result[f'{shot_name}'] = item

100%|██████████| 7475/7475 [00:02<00:00, 3706.29it/s]


In [36]:
result['1']

{'id': '00001',
 'vimeo_id': '96990666',
 'url': 'https://vimeo.com/96990666',
 'title': 'PARISIANA JAM',
 'description': 'On Sunday, May 25th, many riders from Paris and from Argentina came to the infamous "Opera Spot" in the middle of Paris to ride together.\nIt was a great time: we rode as hard as we could, we spoke together, we chilled a lot...\nValentin FLAD - May 2014',
 'channel': 'valentinxflad',
 'tags': ['fish eye',
  'edit',
  'flad',
  'vid?o',
  'acrobatique',
  'Flatland',
  'riding',
  'BMX',
  'canon',
  'v?lo',
  'paris',
  'jam',
  'Flat',
  'france',
  'sport',
  'Opera',
  'Bike',
  'extreme'],
 'categories': ['/categories/sports'],
 'upload_date': '2014-05-31 10:32:38',
 'date': '20140531',
 'month': 'May',
 'year': 2014,
 'day_of_week': 'Saturday',
 'part_of_day': 'morning',
 'local_time': '10:32',
 'width': 1280,
 'height': 720}

In [37]:
len(result)

17235

In [38]:
# convert to dataframe
import pandas as pd
df = pd.DataFrame.from_dict(result, orient='index')
df['index'] = df.index
df.head()

,id,vimeo_id,url,title,description,channel,tags,categories,upload_date,date,month,year,day_of_week,part_of_day,local_time,width,height,index
14693,14693,129402440,https://vimeo.com/129402440,Airplay in Palau,,user23567282,"[Airplay, scuba diving, palau, underwater]","[/categories/travel, /categories/sports]",2015-06-01 01:13:22,20150601,June,2015,Monday,night,01:13,1280,720,14693
12130,12130,231182522,https://vimeo.com/231182522,"Slacklining in Varistonpuisto, Vantaa, Finland...",Beginner level slacklining in public park. Loc...,user24728254,"[vantaa, slacklining, slackline, finland]","[/categories/sports/outdoorsports/videos, /cat...",2017-08-26 05:43:35,20170826,August,2017,Saturday,early morning,05:43,1920,1080,12130
14114,14114,123250553,https://vimeo.com/123250553,Rick Braun,,caseyacaster,"[Rick Braun, music, music doc, jazz, smooth ja...","[/categories/music, /categories/documentary]",2015-03-25 17:55:06,20150325,March,2015,Wednesday,early evening,17:55,1920,1080,14114
7848,07848,103994354,https://vimeo.com/103994354,"Hotel Villa Pigna - Ascoli Piceno, Marche. Italy","So easy, so friendly.\nMaggiori informazioni s...",hotelvillapigna,"[Marche, Ascoli Piceno, Hotel, Villa Pigna]","[/categories/travel, /categories/personal, /ca...",2014-08-21 06:30:02,20140821,August,2014,Thursday,early morning,06:30,1280,720,7848
9693,09693,29606634,https://vimeo.com/29606634,Dream - 25 Years Celebration of Baizid Steel,"This video was shot in Chittagong, Bangladesh,...",augustrock,"[factory, mills, Trance, 5D Mark II, dangerous...","[/categories/hd/canon/videos, /categories/narr...",2011-09-26 10:05:26,20110926,September,2011,Monday,morning,10:05,1280,720,9693


In [39]:
df.to_csv('vbs22_meta.csv', index=False)

In [40]:
tmp_df = df[['id','url']]
# tmp_df.to_dict(orient='id')
tmp_dict = {str(k): str(v['url']) for k, v in tmp_df.to_dict(orient='id').items()}
tmp_dict

/tmp/ipykernel_3518019/1934795079.py:3: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  tmp_dict = {str(k): str(v['url']) for k, v in tmp_df.to_dict(orient='id').items()}


{'14693': 'https://vimeo.com/129402440',
 '12130': 'https://vimeo.com/231182522',
 '14114': 'https://vimeo.com/123250553',
 '7848': 'https://vimeo.com/103994354',
 '9693': 'https://vimeo.com/29606634',
 '13538': 'https://vimeo.com/6528658',
 '9234': 'https://vimeo.com/204663669',
 '9923': 'https://vimeo.com/30419409',
 '14029': 'https://vimeo.com/37447598',
 '14460': 'https://vimeo.com/36933268',
 '12386': 'https://vimeo.com/111033995',
 '12608': 'https://vimeo.com/181555628',
 '8757': 'https://vimeo.com/72502272',
 '13950': 'https://vimeo.com/62994665',
 '17197': 'https://vimeo.com/189102997',
 '12452': 'https://vimeo.com/86946713',
 '11963': 'https://vimeo.com/41955573',
 '13742': 'https://vimeo.com/29324764',
 '16023': 'https://vimeo.com/123268587',
 '12666': 'https://vimeo.com/110622545',
 '16758': 'https://vimeo.com/80306910',
 '8161': 'https://vimeo.com/37327405',
 '9585': 'https://vimeo.com/103581762',
 '15935': 'https://vimeo.com/27801049',
 '13527': 'https://vimeo.com/18594240

In [41]:
# save to json (optional)
# import json
# with open('vbs22_meta.json', 'w') as f:
#     json.dump(tmp_dict, f)